In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, utils
from torchvision.transforms import ToTensor, ToPILImage, Lambda
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
import torch.optim as optim
import numpy as np
import torchvision.transforms as tt

In [3]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), tt.RandomHorizontalFlip(), tt.ToTensor(), tt.Normalize(*stats, inplace=True)])
valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

In [4]:
# Imports the training dataset
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=train_tfms,
    target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

# Imports the testing dataset
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=valid_tfms,
)

Files already downloaded and verified
Files already downloaded and verified


In [48]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 8, shuffle = True)

In [49]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [69]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.ConvStack1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
            )
        self.ResStack1 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            )
        self.ConvStack2 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
            )
        self.ResStack2 = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            )
        self.MaxPool = nn.AvgPool2d(4)
        self.Flatten = nn.Flatten()
        self.Linear = nn.Linear(512, 10)

    def forward(self, x):
        x = self.ConvStack1(x)
        x = self.ResStack1(x) + x
        x = self.ConvStack2(x)
        x = self.ResStack2(x) + x
        x = self.MaxPool(x)
        x = self.Flatten(x)
        x = self.Linear(x)

        # x = x.reshape(x.shape[0], -1)
        # print(x.shape)
        # print(x)
        return x

In [70]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (ConvStack1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Dropout(p=0.1, inplace=False)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ResStack1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (ConvStack2): Sequential(
    (

In [71]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [76]:
# number of epochs to train the model
n_epochs = 5
#List to store loss to visualize
train_losslist = []
test_losslist = []
test_loss_min = 0.311371 # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    test_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        if device == "cuda":
            inputs, labels = inputs.cuda(), labels.cuda()
        optimiser.zero_grad()
        if device == "cuda":
            model.cuda()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimiser.step()
        # update training loss
        train_loss += loss.item()*inputs.size(0)
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'Training - [{epoch}, {i + 1:5d}] loss: {train_loss / 1000:.3f}')

        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data
        if device == "cuda":
            inputs, labels = inputs.cuda(), labels.cuda()
        output = model(inputs)
        loss = criterion(output, labels)
        # update average test loss 
        test_loss += loss.item()*inputs.size(0)
        if i % 1000 == 999:    # print every 1000 mini-batches
            print(f'Testing - [{epoch}, {i + 1:5d}] loss: {train_loss / 1000:.3f}')
    
    # calculate average loss
    train_loss = train_loss/len(train_dataloader.dataset)
    test_loss = test_loss/len(test_dataloader.dataset)
    # put loss in list, to be shown in graph
    train_losslist.append(train_loss)
    test_losslist.append(test_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch, train_loss, test_loss))
    
    # save model if validation loss has decreased
    if test_loss <= test_loss_min:
        print('New best model found ({:.6f} --> {:.6f}), saving'.format(
        test_loss_min,
        test_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        test_loss_min = test_loss

Training - [1,  1000] loss: 1.917
Training - [1,  2000] loss: 3.732
Training - [1,  3000] loss: 5.656
Training - [1,  4000] loss: 7.503
Training - [1,  5000] loss: 9.411
Training - [1,  6000] loss: 11.353
Testing - [1,  1000] loss: 11.826
Epoch: 1 	Training Loss: 0.236527 	Test Loss: 0.282354
New best model found (0.311371 --> 0.282354), saving
Training - [2,  1000] loss: 1.799
Training - [2,  2000] loss: 3.608
Training - [2,  3000] loss: 5.489
Training - [2,  4000] loss: 7.335
Training - [2,  5000] loss: 9.248
Training - [2,  6000] loss: 11.097
Testing - [2,  1000] loss: 11.625
Epoch: 2 	Training Loss: 0.232505 	Test Loss: 0.311069
Training - [3,  1000] loss: 1.754
Training - [3,  2000] loss: 3.603
Training - [3,  3000] loss: 5.314
Training - [3,  4000] loss: 7.008
Training - [3,  5000] loss: 8.733
Training - [3,  6000] loss: 10.398
Testing - [3,  1000] loss: 10.878
Epoch: 3 	Training Loss: 0.217567 	Test Loss: 0.323338
Training - [4,  1000] loss: 1.759
Training - [4,  2000] loss: 3.2

In [77]:
model.load_state_dict(torch.load('model_cifar.pt'))
model.eval()

NeuralNetwork(
  (ConvStack1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Dropout(p=0.1, inplace=False)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ResStack1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (ConvStack2): Sequential(
    (

In [78]:
labels_map = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [79]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = True
model.eval()
# iterate over test data
for data, target in test_dataloader:
    # move tensors to GPU if CUDA is available
    data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(4):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_dataloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            labels_map[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (labels_map[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.282354

Test Accuracy of plane: 91% (464/509)
Test Accuracy of   car: 97% (484/498)
Test Accuracy of  bird: 88% (430/487)
Test Accuracy of   cat: 81% (396/485)
Test Accuracy of  deer: 93% (473/505)
Test Accuracy of   dog: 85% (413/484)
Test Accuracy of  frog: 87% (429/493)
Test Accuracy of horse: 93% (473/507)
Test Accuracy of  ship: 94% (496/524)
Test Accuracy of truck: 95% (485/508)

Test Accuracy (Overall): 90% (4543/5000)
